In [2]:
import torch
from transformers import GPT2Tokenizer, T5ForConditionalGeneration 

In [109]:
test_examples = {
    'я купил iphone 12X за 142 990 руб без 3-x часов полдень и т.д.': 
        "я купил айфон двенадцать икс за сто сорок две тысячи девятьсот девяносто руб без трёх часов полдень и т.д.",
    'я купил айфон за 14 970 рублей': 
        "я купил айфон за четырнадцать тысяч девятьсот семьдесят рублей",
    "Временами я думаю, какое применение найти тем 14 697 рублям, что лежат уже больше 33 лет?": 
        "Временами я думаю, какое применение найти тем четырнадцати тысячам шестистам девяносто семи рублям, что лежат уже больше тридцати трёх лет?",
    "Было у отца 3 сына, но не было даже 2-3 пиджаков с блёстками за 142 990 рублей.": 
        "Было у отца три сына, но не было даже двух-трёх пиджаков с блёстками за сто сорок две тысячи девятьсто девяносто рублей.",
    "В школе у меня одни 5.": 
        "В школе у меня одни пятёрки.",
    'Было у отца 3 сына. Старшему было 35, среднему - не меньше 33, а младший на 4 младше всех. Бывает.': 
        "Было у отца три сына. Старшему было тридцать пять, среднему - не меньше тридцати трех, а младший на четыре младше всех. Бывает.",
}

In [1]:
import re


# re_tokens = re.compile(r"[а-яА-Я]+\s*|\d+(?:\.\d+)?\s*|[^а-яА-Я\d\s]+\s*")
re_tokens = re.compile(r"(?:[.,!?]|[а-яА-Я]\S*|\d\S*(?:\.\d+)?|[^а-яА-Я\d\s]+)\s*")


def tokenize(text):
    return re.findall(re_tokens, text)


def strip_numbers(s):
    return " ".join(((" ".join(part) if part.isdigit() else part) for part in s.split()))


def strip_numbers(s):
    result = []
    for part in s.split():
        if part.isdigit():
            while len(part) > 3:
                result.append(part[:- 3 * ((len(part) - 1) // 3)])
                part = part[- 3 * ((len(part) - 1) // 3):]
            if part:
                result.append(part)
        else:
            result.append(part)
    return " ".join(result)


def construct_prompt(text):
    result = "<SC1>"
    etid = 0
    token_to_add = ""
    for token in tokenize(text) + [""]:
        if not re.search("\d", token):
            if token_to_add:
                end_match = re.search(r"(.+?)(\W+)$", token_to_add, re.M).groups()
                result += f"[{strip_numbers(end_match[0])}]<extra_id_{etid}>{end_match[1]}"
                etid += 1
                token_to_add = ""
            result += token
        else:
            token_to_add += token
    return result


construct_prompt('я купил iphone 12X за 142 990 руб без 3-x часов 12:00, и т.д.')

'<SC1>я купил iphone [12X]<extra_id_0> за [142 990]<extra_id_1> руб без [3-x]<extra_id_2> часов [12:00]<extra_id_3>, и т.д.'

In [90]:
import re


def construct_answer(prompt:str, prediction:str) -> str:
    replaces = []
    re_prompt = re.compile(r"\[([^\]]+)\]<extra_id_(\d+)>")
    re_pred = re.compile(r"\<extra_id_(\d+)\>(.+?)(?=\<extra_id_\d+\>|</s>)")
    pred_data = {}
    for match in re.finditer(re_pred, prediction.replace("\n", " ")):
        pred_data[match[1]] = match[2].strip()
    while match := re.search(re_prompt, prompt):
        replace = pred_data.get(match[2], match[1])
        prompt = prompt[:match.span()[0]] + replace + prompt[match.span()[1]:]
    return prompt.replace("<SC1>", "")
        
construct_answer(
    '<SC1>Было у отца [3]<extra_id_0> сына. Старшему было [35]<extra_id_1>, среднему - не меньше [33]<extra_id_2>, а младший на [4]<extra_id_3> младше всех. Бывает.',
    """<extra_id_0>  три
 <extra_id_1>  тридцать пять
 <extra_id_2>  тридцати трех
 <extra_id_3>  четыре
</s>"""
)

'Было у отца три сына. Старшему было тридцать пять, среднему - не меньше тридцати трех, а младший на четыре младше всех. Бывает.'

## FRED-T5-large-FT

In [1]:
device='cuda:1'

In [3]:
# path = "./5_fred-t5_librusec/checkpoint-2000/"
# path = "/home/jovyan/wdc1/models/FRED-T5-large"
# path = "4_fred-t5_librusec_final"
path = "6_fred-t5_ficbook/checkpoint-4500/"
path = "/home/jovyan/work/models/3_fred-t5/checkpoint-11000"
tokenizer = GPT2Tokenizer.from_pretrained(path, eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained(path).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [74]:
def predict(text):
    input_ids = torch.tensor([tokenizer.encode(text)]).to(device)
    outputs = model.generate(input_ids, eos_token_id=tokenizer.eos_token_id, early_stopping=True)
    return tokenizer.decode(outputs[0][1:])

In [75]:
predict("<SC1>Были у отца штаны [XXL]<extra_id_0> размера.")

'<extra_id_0>  двадцатого\n</s>'

In [110]:
for lm_text, gt in test_examples.items():
    prompt = construct_prompt(lm_text)
    pred = construct_answer(prompt, predict(prompt))
    if gt == pred:
        print(f"{gt}\n")
    else:
        print(f"{lm_text}\n{prompt}\n{gt}\n{pred}\n")

я купил iphone 12X за 142 990 руб без 3-x часов полдень и т.д.
<SC1>я купил iphone [12X]<extra_id_0> за [142 990]<extra_id_1> руб без [3-x]<extra_id_2> часов полдень и т.д.
я купил айфон двенадцать икс за сто сорок две тысячи девятьсот девяносто руб без трёх часов полдень и т.д.
я купил iphone двенадцатый за сто сорок две тысячи девятьсот девяносто руб без трех часов полдень и т.д.

я купил айфон за четырнадцать тысяч девятьсот семьдесят рублей

Временами я думаю, какое применение найти тем четырнадцати тысячам шестистам девяносто семи рублям, что лежат уже больше тридцати трёх лет?

Было у отца 3 сына, но не было даже 2-3 пиджаков с блёстками за 142 990 рублей.
<SC1>Было у отца [3]<extra_id_0> сына, но не было даже [2-3]<extra_id_1> пиджаков с блёстками за [142 990]<extra_id_2> рублей.
Было у отца три сына, но не было даже двух-трёх пиджаков с блёстками за сто сорок две тысячи девятьсто девяносто рублей.
Было у отца три сына, но не было даже двух- трёх пиджаков с блёстками за сто соро